In [1]:
from __future__ import print_function
from unicodecsv import DictReader
import pandas as pd
from paths import DATASETS_DIR, DUMPS_DIR
from pprint import pprint
from data import generate_dump
import os
from collections import OrderedDict
# import mysql.connector
from sqlalchemy import create_engine
# import pyodbc

In [2]:
dump_path = os.path.join(DUMPS_DIR, "series-tiempo.{}")

## Genero dataframe con el dump

In [3]:
%time df = generate_dump()

Distribución 19.2 no tiene field indice_tiempo
CPU times: user 26.1 s, sys: 1.47 s, total: 27.6 s
Wall time: 27.6 s


In [4]:
drop_cols = ["distribucion_indice_tiempo", "valor"]
index_cols = ["dataset_id", "distribucion_id", "serie_id"]
series_cols = df.columns.drop(drop_cols)
series_group = df.groupby(index_cols)

In [5]:
df_series = df.drop(drop_cols, axis=1).drop_duplicates().set_index(index_cols)

In [37]:
df_series["distribucion_indice_inicio"] = series_group["distribucion_indice_tiempo"].min()
df_series["distribucion_indice_final"] = series_group["distribucion_indice_tiempo"].max()
df_series["valor_ultimo"] = series_group.apply(lambda x: x.loc[x.distribucion_indice_tiempo.argmax(), "valor"])
df_series["valor_anterior"] = series_group.apply(
    lambda x: pd.Series(list(x.valor), list(x.distribucion_indice_tiempo))[-2]
)
df_series["var_pct_anterior"] = series_group.apply(
    lambda x: pd.Series(list(x.valor), list(x.distribucion_indice_tiempo)).pct_change()[-1]
)

In [38]:
df_series.head()

distribucion_indice_frecuencia  \
dataset_id distribucion_id serie_id                                              
1          1.1             1.1_OGI_D_1993_A_25                           R/P1Y   
                           1.1_DGCP_D_1993_A_27                          R/P1Y   
                           1.1_DGI_D_1993_A_19                           R/P1Y   
                           1.1_OGP_D_1993_A_17                           R/P1Y   
                           1.1_DGCP_D_1993_A_30                          R/P1Y   

                                                                   serie_titulo  \
dataset_id distribucion_id serie_id                                               
1          1.1             1.1_OGI_D_1993_A_25        oferta_global_importacion   
                           1.1_DGCP_D_1993_A_27     demanda_global_consumo_priv   
                           1.1_DGI_D_1993_A_19              demanda_global_ibif   
                           1.1_OGP_D_1993_A_17                oferta_global_pbi   
                           1.1_DGCP_D_1993_A_30  demanda_global_consumo_publico   

                                                                      serie_unidades  \
dataset_id distribucion_id serie_id                                                    
1          1.1             1.1_OGI_D_1993_A_25   Millones de pesos a precios de 1993   
                           1.1_DGCP_D_1993_A_27  Millones de pesos a precios de 1993   
                           1.1_DGI_D_1993_A_19   Millones de pesos a precios de 1993   
                           1.1_OGP_D_1993_A_17   Millones de pesos a precios de 1993   
                           1.1_DGCP_D_1993_A_30  Millones de pesos a precios de 1993   

                                                                                 serie_descripcion  \
dataset_id distribucion_id serie_id                                                                  
1          1.1             1.1_OGI_D_1993_A_25   Importaciones desestacionalizadas, en millones...   
                           1.1_DGCP_D_1993_A_27  Consumo privado desestacionalizado, en millone...   
                           1.1_DGI_D_1993_A_19   Inversion bruta interna fija desestacionalizad...   
                           1.1_OGP_D_1993_A_17   PIB desestacionalizado, en millones de pesos d...   
                           1.1_DGCP_D_1993_A_30  Consumo publico desestacionalizado, en millone...   

                                                                               distribucion_titulo  \
dataset_id distribucion_id serie_id                                                                  
1          1.1             1.1_OGI_D_1993_A_25   Oferta y Demanda Globales a precios de 1993: D...   
                           1.1_DGCP_D_1993_A_27  Oferta y Demanda Globales a precios de 1993: D...   
                           1.1_DGI_D_1993_A_19   Oferta y Demanda Globales a precios de 1993: D...   
                           1.1_OGP_D_1993_A_17   Oferta y Demanda Globales a precios de 1993: D...   
                           1.1_DGCP_D_1993_A_30  Oferta y Demanda Globales a precios de 1993: D...   

                                                                          distribucion_descripcion  \
dataset_id distribucion_id serie_id                                                                  
1          1.1             1.1_OGI_D_1993_A_25   Oferta y Demanda Globales por componente, a pr...   
                           1.1_DGCP_D_1993_A_27  Oferta y Demanda Globales por componente, a pr...   
                           1.1_DGI_D_1993_A_19   Oferta y Demanda Globales por componente, a pr...   
                           1.1_OGP_D_1993_A_17   Oferta y Demanda Globales por componente, a pr...   
                           1.1_DGCP_D_1993_A_30  Oferta y Demanda Globales por componente, a pr...   

                                                                               dataset_responsable  \
dataset_id distribucion_id 

In [33]:
series_partial = df[df.serie_id == "1.2_OGP_D_1993_T_17"]

In [21]:
pd.infer_freq(series_partial.distribucion_indice_tiempo)

'AS-JAN'

In [ ]:
rng = pd.date_range()

In [27]:
series = pd.Series(list(series_partial.valor), list(series_partial.distribucion_indice_tiempo))

In [32]:
series.pct_change(1)

1993-01-01         NaN
1994-01-01    0.211310
1995-01-01   -0.098066
1996-01-01    0.175291
1997-01-01    0.268716
1998-01-01    0.084139
1999-01-01   -0.112668
2000-01-01   -0.001576
2001-01-01   -0.139466
2002-01-01   -0.500596
2003-01-01    0.375654
2004-01-01    0.401218
2005-01-01    0.201388
2006-01-01    0.153755
2007-01-01    0.204925
2008-01-01    0.140940
2009-01-01   -0.190254
2010-01-01    0.339764
2011-01-01    0.177956
2012-01-01   -0.055679
dtype: float64

In [14]:
series_partial.distribucion_indice_tiempo.argmax()

114

In [7]:
df_series

distribucion_indice_frecuencia  \
dataset_id distribucion_id serie_id                                                 
1          1.1             1.1_OGI_D_1993_A_25                              R/P1Y   
                           1.1_DGCP_D_1993_A_27                             R/P1Y   
                           1.1_DGI_D_1993_A_19                              R/P1Y   
                           1.1_OGP_D_1993_A_17                              R/P1Y   
                           1.1_DGCP_D_1993_A_30                             R/P1Y   
                           1.1_DGE_D_1993_A_26                              R/P1Y   
           1.2             1.2_OGI_D_1993_T_25                              R/P3M   
                           1.2_DGCP_D_1993_T_27                             R/P3M   
                           1.2_DGI_D_1993_T_19                              R/P3M   
                           1.2_OGP_D_1993_T_17                              R/P3M   
                           1.2_DGCP_D_1993_T_30                             R/P3M   
                           1.2_DGE_D_1993_T_26                              R/P3M   
2          2.1             2.1_OGI_1993_A_25                                R/P1Y   
                           2.1_DGIEDP_1993_A_45                             R/P1Y   
                           2.1_DGT_1993_A_20                                R/P1Y   
                           2.1_DGIEDPMET_1993_A_51                          R/P1Y   
                           2.1_DGIEDPI_1993_A_55                            R/P1Y   
                           2.1_DGI_1993_A_19                                R/P1Y   
                           2.1_DGIC_1993_A_32                               R/P1Y   
                           2.1_DGCP_1993_A_30                               R/P1Y   
                           2.1_DGIEDPMTN_1993_A_56                          R/P1Y   
                           2.1_DGCP_1993_A_27                               R/P1Y   
                           2.1_OGP_1993_A_17                                R/P1Y   
                           2.1_OGT_1993_A_19                                R/P1Y   
                           2.1_DGIEDPMTI_1993_A_57                          R/P1Y   
                           2.1_DGIEDPMTT_1993_A_53                          R/P1Y   
                           2.1_DEVE_1993_A_46                               R/P1Y   
                           2.1_DGE_1993_A_26                                R/P1Y   
                           2.1_DGIEDPMEI_1993_A_55                          R/P1Y   
                           2.1_DGIEDPN_1993_A_54                            R/P1Y   
...                                                                           ...   
164        164.2           164.2_SATERIANCP_0_0_57                          R/P3M   
                           164.2_SAL_AZUTOS_0_0_50                          R/P3M   
                           164.2_SRUTAS_LES_0_0_28                          R/P3M   
                           164.2_SEREALELES_0_0_11                          R/P3M   
                           164.2_SIDRIO_RAS_0_0_22                          R/P3M   
                           164.2_SRODUCTCOS_0_0_42                          R/P3M   
                           164.2_SANUFACSAS_0_0_24                          R/P3M   
                           164.2_SREPARARIA_0_0_57                          R/P3M   
                           164.2_SEDAEDA_0_0_7                              R/P3M   
                           164.2_SAQUINATES_0_0_46                          R/P3M   
                           164.2_SEMAS_MRAS_0_0_37                          R/P3M   
                           164.2_SRMAS_MIOS_0_0_37                          R/P3M   
                           164.2_SRODUCTDON_0_0_36                          R/P3M   
                           164.2_SEMAS_PNCP_0_0_36                          R/P3M   
                           164.2_SERONAVTES_0_0_40      

In [ ]:
df_series.to_excel("series-tiempo-resumen.xlsx", "resumen", index=True, index_label=True, merge_cells=False)

## Genero distribuciones de distintos formatos

In [ ]:
%%time
# CSV
path = dump_path.format("csv")
df.to_csv(path, encoding="utf-8", sep=",", index=False)
print("{}MB".format(os.path.getsize(path) / 1000000))

In [ ]:
%%time 
# EXCEL
path = dump_path.format("xlsx")
writer = pd.ExcelWriter(path, engine='xlsxwriter')
df.to_excel(writer, "data", merge_cells=False, encoding="utf-8", index=False)
writer.save()
print("{}MB".format(os.path.getsize(path) / 1000000))

In [ ]:
%%time 
# SQLITE
path = dump_path.format("db")
engine = create_engine('sqlite:///{}'.format(path), echo=True)
df.to_sql("series_tiempo", engine, index=False, if_exists="replace")
print("{}MB".format(os.path.getsize(path) / 1000000))

In [ ]:
%%time 
# ACCESS
# set up some constants
DRV = '{Microsoft Access Driver (*.mdb)}'; PWD = 'pw'

# connect to db
path = dump_path.format("mdb")
con = pyodbc.connect('DRIVER={};DBQ={};PWD={}'.format(DRV, path, PWD))
cur = con.cursor()

df.to_sql("series_tiempo", con, index=False, if_exists="replace")
print("{}MB".format(os.path.getsize(path) / 1000000))

In [ ]:
%%time 
# SQL
# la gracia acá es hacer bien las tablas con sus índices y pensar si MySQL, Postgresql o SQLite
path = dump_path.format("sql")
df.to_sql(path, "data", merge_cells=False, encoding="utf-8", index=False)
print("{}MB".format(os.path.getsize(path) / 1000000))

In [ ]:
%%time 
# DTA
# hay que ver cómo manejar el límite de caracteres
path = dump_path.format("dta")
df.to_stata(path, write_index=False)
print("{}MB".format(os.path.getsize(path) / 1000000))

In [ ]:
df.head()